Importing the necessary libraries

In [1]:
import requests
import pandas as pd
import lxml.html as lh

Defining the url of the Wikipedia page and groom for data scraping

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Filter the different items from the table and add them to a dictionary to create the table

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [5]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [6]:
[len(C) for (title,C) in col]

[289, 289, 289]

In [7]:
Dict={title:column for (title,column) in col}
df = pd.DataFrame.from_dict(Dict)

Drop rows that have "Not assigned" in the Borough column

In [8]:
df = df[df.Borough != "Not assigned"]

Rename Neighbourhood column

In [9]:
df.rename(index=str, columns={"Neighbourhood\n": "Neighbourhood"}, inplace=True)

Make the "Not assigned" values in the Neighbourhood column equal to the Borough column

In [10]:
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned\n', df.Neighbourhood)

Group by Postcode and Join the values in Neighbourhood

In [11]:
df = df.groupby('Postcode', as_index=False).agg({'Borough':'first', 'Neighbourhood':', '.join})

Remove unwaned \n characters

In [12]:
df = df.replace(r'\n',' ', regex=True) 

Rename Postcode column

In [13]:
df.rename(index=str, columns={"Postcode": "PostalCode"}, inplace=True)

In [14]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get dataframe shape

In [15]:
df.shape

(103, 3)

Get the Geo Data, since the method described in the assignment didn't work for me, but it said using the csv was ok.

In [16]:
location = pd.read_csv('https://cocl.us/Geospatial_data')

In [17]:
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Converting and Renaming Columns

In [18]:
location.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace=True)

In [19]:
location['Latitude'] = location['Latitude'].astype(str)

In [20]:
location['Longitude'] = location['Longitude'].astype(str)

Merge the Dataframes

In [21]:
df_complete = pd.merge(df,
                 location[['PostalCode', 'Latitude', 'Longitude']],
                 on='PostalCode')

In [22]:
df_complete.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.8066863,-79.1943534
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.7845351,-79.1604971
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761
